In [2]:
#!pip show tensorflow_datasets
!pip3 install emnist
!pip3 install tensorflow_datasets
!pip3 install opencv_python
!pip3 install matplotlib
!pip3 install chardet
!pip3 install pandas

In [3]:
# import emnist
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow_datasets as tfds
import tensorflow.keras as keras
import sklearn
from sklearn.decomposition import PCA 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
import cv2
from tensorflow.keras.layers import Input, Lambda, Conv2D,Conv2DTranspose, MaxPooling2D, BatchNormalization, Dense, Flatten, Activation, Dropout
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import backend as K
%matplotlib inline
# tf.debugging.set_log_device_placement(False)

In [4]:
print(tf.config.list_physical_devices('GPU'))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
print(tf.test.is_built_with_cuda())

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Default GPU Device: /device:GPU:0
True


In [5]:
(ds_train, ds_test), info = tfds.load('omniglot', split=['train', 'test'], with_info=True)

In [6]:
df_train = tfds.as_dataframe(ds_train, info)
df_test  = tfds.as_dataframe(ds_test, info)

In [7]:
train_images = np.stack(df_train['image'])
train_images.shape

(19280, 105, 105, 3)

In [8]:
df_train.columns

Index(['alphabet', 'alphabet_char_id', 'image', 'label'], dtype='object')

In [9]:
len(df_train['label'].unique())

964

In [10]:
#df_train[['alphabet_char_id', 'label']].loc[np.where((df_train['alphabet'] == 27) & (df_train['alphabet_char_id'] == 23))]

In [11]:
''' Data handling general functions '''

def separate_fewshot(test_images, test_labels, n=1):
    oneshot_data = []
    classify_data = []
    for label in np.unique(test_labels):
        for num in np.random.choice(np.where(test_labels == label)[0], n, False):
            oneshot_data.append(num)
    temp = set(oneshot_data)
    for i in range(len(test_labels)):
        if not i in temp: classify_data.append(i)
    oneshot_images = test_images[oneshot_data]
    oneshot_labels = test_labels[oneshot_data]
    classify_images = test_images[classify_data]
    classify_labels = test_labels[classify_data]
    return oneshot_images, oneshot_labels, classify_images, classify_labels

In [12]:
def resize_images(images, size, to_grayscale = True):
    resized_images = []
    for img in images:
        resized_image = cv2.resize(img, (size, size))
        if to_grayscale:
            resized_image= cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
        resized_images.append(resized_image)
    return np.array(resized_images)

In [13]:
def parse_omniglot_dataframe(df, img_size = 56, reshape=True):
    images = resize_images(df['image'], img_size)
    if reshape:
        images = images.reshape(-1, img_size * img_size)
    else:
        images.reshape(-1, img_size, img_size)
        images = images[..., np.newaxis]
    labels = df['label'].to_numpy()
    return (images, labels)

In [14]:
def get_mini_dataset(
        self, batch_size, repetitions, shots, num_classes, split=False
    ):
        temp_labels = np.zeros(shape=(num_classes * shots))
        temp_images = np.zeros(shape=(num_classes * shots, 28, 28, 1))
        if split:
            test_labels = np.zeros(shape=(num_classes))
            test_images = np.zeros(shape=(num_classes, 28, 28, 1))

        # Get a random subset of labels from the entire label set.
        label_subset = random.choices(self.labels, k=num_classes)
        for class_idx, class_obj in enumerate(label_subset):
            # Use enumerated index value as a temporary label for mini-batch in
            # few shot learning.
            temp_labels[class_idx * shots : (class_idx + 1) * shots] = class_idx
            # If creating a split dataset for testing, select an extra sample from each
            # label to create the test dataset.
            if split:
                test_labels[class_idx] = class_idx
                images_to_split = random.choices(
                    self.data[label_subset[class_idx]], k=shots + 1
                )
                test_images[class_idx] = images_to_split[-1]
                temp_images[
                    class_idx * shots : (class_idx + 1) * shots
                ] = images_to_split[:-1]
            else:
                # For each index in the randomly selected label_subset, sample the
                # necessary number of images.
                temp_images[
                    class_idx * shots : (class_idx + 1) * shots
                ] = random.choices(self.data[label_subset[class_idx]], k=shots)

        dataset = tf.data.Dataset.from_tensor_slices(
            (temp_images.astype(np.float32), temp_labels.astype(np.int32))
        )
        dataset = dataset.shuffle(100).batch(batch_size).repeat(repetitions)
        if split:
            return dataset, test_images, test_labels
        return dataset

In [15]:
img, lbl = parse_omniglot_dataframe(df_test, reshape=False)
img.shape

(13180, 56, 56, 1)

In [16]:
matches = 0
total = 0
img_size = 56
verbose = True
(train_images, train_labels) = parse_omniglot_dataframe(df_train, img_size)
(test_images, test_labels) = parse_omniglot_dataframe(df_test, img_size)
t_alphabets = df_test['alphabet'].to_numpy()

for alphabet in np.unique(t_alphabets):
    ind_alphabet = np.where(t_alphabets == alphabet)[0]
    labels = test_labels[ind_alphabet]
    images = test_images[ind_alphabet]
    os_img, os_label, clas_img, clas_label = separate_fewshot(images, labels, n=1)

    #os_img = encoder.predict(os_img)
    #clas_img = encoder.predict(clas_img)

    #if verbose: print("Learning oneshot ...")
    nn = 1
    neigh = KNeighborsClassifier(n_neighbors = nn)
    neigh.fit(os_img, os_label)

    #if verbose: print("Predicting ...")
    pred = neigh.predict(clas_img)

    matches += np.sum(pred == clas_label)
    total += len(clas_label)

if verbose:
    print("Accuracy: ", matches/total)
    print(f"======= DO NOTHING method: Finished =======")

print(matches/total)

Accuracy:  0.1146074594680936
======= DO NOTHING method: Finished =======
0.1146074594680936


In [17]:
def test_PCA(df_train, df_test, img_size=56, n=1, n_components = 32, verbose=False, train=1):
    
    (train_images, train_labels) = parse_omniglot_dataframe(df_train, img_size)
    (test_images, test_labels) = parse_omniglot_dataframe(df_test, img_size)
    t_alphabets = df_test['alphabet'].to_numpy()
    
    if verbose: print("======= PCA method: Training and evaluating ... =======")
    if verbose: print("Learning background ...")
    pca = PCA(n_components=n_components)
    pca.fit(X=train_images)
    
    matches = 0
    total = 0
    
    if verbose: print("Vectorizing ...")
    for alphabet in np.unique(t_alphabets):
        ind_alphabet = np.where(t_alphabets == alphabet)[0]
        labels = test_labels[ind_alphabet]
        images = test_images[ind_alphabet]
        os_img, os_label, clas_img, clas_label = separate_fewshot(images, labels, n=n)
        
        
        os_img = pca.transform(os_img)
        clas_img = pca.transform(clas_img)

        #if verbose: print("Learning oneshot ...")
        nn = min(train, 5)
        neigh = KNeighborsClassifier(n_neighbors = nn)
        neigh.fit(os_img, os_label)

        #if verbose: print("Predicting ...")
        pred = neigh.predict(clas_img)
        
        matches += np.sum(pred == clas_label)
        total += len(clas_label)
        
    if verbose:
        print("Accuracy: ", matches/total)
        print("======= PCA method: Finished =======")

    return matches/total

In [18]:
test_PCA(df_train, df_test, verbose=True)

======= PCA method: Training and evaluating ... =======
Learning background ...
Vectorizing ...
Accuracy:  0.17067326890823417
======= PCA method: Finished =======


0.17067326890823417

In [19]:
def test_LDA(df_train, df_test, img_size=56, n=1, n_components = 32, verbose=False, train=1, c=3):
    
    (train_images, train_labels) = parse_omniglot_dataframe(df_train, img_size)
    (test_images, test_labels) = parse_omniglot_dataframe(df_test, img_size)
    t_alphabets = df_test['alphabet'].to_numpy()
    
    # unique_labels = df_train['label'].unique()
    # subsample_index = []
    # for label in unique_labels:
    #     for ind in np.random.choice(np.where(df_train['label'] == label)[0], c, False):
    #         subsample_index.append(ind)
    # subsample_index = np.array(subsample_index)
    # train_images = train_images[subsample_index]
    # train_labels = train_labels[subsample_index]
    
    if verbose: print("======= LDA method: Training and evaluating ... =======")
    if verbose: print("Learning background ...")
    lda = LDA(n_components=n_components)
    lda.fit(X=train_images,y=train_labels)
    
    matches = 0
    total = 0
    
    if verbose: print("Vectorizing ...")
    for alphabet in np.unique(t_alphabets):
        ind_alphabet = np.where(t_alphabets == alphabet)[0]
        labels = test_labels[ind_alphabet]
        images = test_images[ind_alphabet]
        os_img, os_label, clas_img, clas_label = separate_fewshot(images, labels, n=n)
        
        
        os_img = lda.transform(os_img)
        clas_img = lda.transform(clas_img)

        #if verbose: print("Learning oneshot ...")
        nn = min(train, 5)
        neigh = KNeighborsClassifier(n_neighbors = nn)
        neigh.fit(os_img, os_label)

        #if verbose: print("Predicting ...")
        pred = neigh.predict(clas_img)
        
        matches += np.sum(pred == clas_label)
        total += len(clas_label)
        
    if verbose:
        print("Accuracy: ", matches/total)
        print("======= LDA method: Finished =======")

    return matches/total

In [20]:
test_LDA(df_train, df_test, verbose=True)

======= LDA method: Training and evaluating ... =======
Learning background ...
Vectorizing ...
Accuracy:  0.15813433431834517
======= LDA method: Finished =======


0.15813433431834517

In [21]:
def nonlinear_autoencoder(input_shape, code_size: int):
    """
    Instanciate and compiles an autoencoder, returns both the autoencoder and just the encoder
    """
    input_size = input_shape[0] * input_shape[1] * input_shape[2]
    encoder = keras.Sequential([
        Flatten(),
        keras.layers.Dense(input_size//4, activation='ReLU'),
        keras.layers.Dense(input_size//8, activation='ReLU'),
        keras.layers.Dense(code_size),
    ])
    
    decoder = keras.Sequential([
        keras.layers.Dense(input_size//8, activation='ReLU'),
        keras.layers.Dense(input_size//4, activation='ReLU'),
        keras.layers.Dense(input_size),
    ])
    
    inputs = keras.Input(shape=input_shape)
    outputs = decoder(encoder(inputs))
    autoencoder = keras.Model(inputs=inputs, outputs=outputs)
    
    autoencoder.compile(optimizer='Adam', loss='MSE')
    return autoencoder, encoder

In [22]:
def linear_autoencoder(input_shape, code_size: int):
    """
    Instanciate and compiles an autoencoder, returns both the autoencoder and just the encoder
    """
    input_size = input_shape[0] * input_shape[1] * input_shape[2]
    encoder = keras.Sequential([
        Flatten(),
        keras.layers.Dense(code_size),
    ])
    
    decoder = keras.Sequential([
        keras.layers.Dense(input_size),
    ])
    
    inputs = keras.Input(shape=input_shape)
    outputs = decoder(encoder(inputs))
    autoencoder = keras.Model(inputs=inputs, outputs=outputs)
    
    autoencoder.compile(optimizer='Adam', loss='MSE')
    return autoencoder, encoder

In [24]:
def cnn_autoencoder(input_shape, code_size: int):
    """
    Instanciate and compiles an autoencoder, returns both the autoencoder and just the encoder

    :param int or tuple input_size: shape of the input samples
    :param int code_size: dimension on which to project the original data
    :return: autoencoder, encoder
    """
    print(input_shape, code_size)
    output_size = input_shape[0] * input_shape[1] * input_shape[2]
    inputs = Input(shape=input_shape)
    encoder = tf.keras.Sequential([
        Conv2D(64, (3, 3), activation='relu', padding='same', strides=2),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu', padding='same', strides=2),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Flatten(),
    ])
        
    decoder = tf.keras.Sequential([
        Dense(output_size // 8, activation='relu'),
        Dense(output_size // 4, activation='relu'),
        Dense(output_size // 2, activation='relu'),
        Dense(output_size)
    ])
    autoencoder = Model(inputs=inputs, outputs=decoder(encoder(inputs)), name="autoencoder")
    autoencoder.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.MeanSquaredError())
    encoder = Model(inputs=inputs, outputs=encoder(inputs), name="encoder")
    
    return autoencoder, encoder

In [25]:
def cnn_autoencoder2(input_shape, code_size: int):
    """
    Instanciate and compiles an autoencoder, returns both the autoencoder and just the encoder

    :param int or tuple input_size: shape of the input samples
    :param int code_size: dimension on which to project the original data
    :return: autoencoder, encoder
    """
    
    print(input_shape, code_size)
    encoder = tf.keras.Sequential([
        Conv2D(32, (3, 3), activation = 'relu'),
        MaxPooling2D((2, 2)),
        Conv2D(32, (3, 3), activation = 'relu'),
        BatchNormalization(),
    ])
    
    output_size = input_shape[0] * input_shape[1] * input_shape[2]
    
    inputs = tf.keras.Input(shape=input_shape)
    conv_layer = Conv2D(filters=32, kernel_size=(3, 3), activation="relu")(inputs)
    pool_layer = MaxPooling2D(pool_size=(2, 2))(conv_layer)
    flatten_layer = Flatten()(pool_layer)
    encoder = Dense(code_size)(flatten_layer)
    dense_layer = Dense(output_size // 2)(encoder)
    final_layer = Dense(output_size)(dense_layer)
    
    autoencoder = Model(inputs=inputs, outputs=final_layer, name="autoencoder")
    autoencoder.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.MeanSquaredError())
    
    encoder = Model(inputs=inputs, outputs=encoder, name="encoder")
    
    return autoencoder, encoder

In [26]:
def test_autoencoder(df_train, df_test, autoencoder, img_size=56, num_components=32, n=1, verbose=False, train=1, epochs=50):
    
    (train_images, train_labels) = parse_omniglot_dataframe(df_train, img_size, reshape=False)
    (test_images, test_labels) = parse_omniglot_dataframe(df_test, img_size, reshape=False)
    t_alphabets = df_test['alphabet'].to_numpy()
    
    #print(test_images.shape)
    name = autoencoder.__name__
    if verbose: print(f"======= {name} Autoencoder method: Training and evaluating ... =======")
    if verbose: print("Learning background ...")
    #print(test_images[0].shape)
    autoencoder, encoder = autoencoder(test_images[0].shape, code_size=num_components)
    print(autoencoder.summary())
    autoencoder.fit(x=train_images,y=train_images.reshape(-1, img_size * img_size), epochs=epochs, batch_size=64)
    
    matches = 0
    total = 0
    
    if verbose: print("Vectorizing ...")
    for alphabet in np.unique(t_alphabets):
        ind_alphabet = np.where(t_alphabets == alphabet)[0]
        labels = test_labels[ind_alphabet]
        images = test_images[ind_alphabet]
        os_img, os_label, clas_img, clas_label = separate_fewshot(images, labels, n=n)
        
        os_img = encoder.predict(os_img)
        clas_img = encoder.predict(clas_img)

        #if verbose: print("Learning oneshot ...")
        nn = min(n, 5)
        neigh = KNeighborsClassifier(n_neighbors = nn)
        neigh.fit(os_img, os_label)

        #if verbose: print("Predicting ...")
        pred = neigh.predict(clas_img)
        
        matches += np.sum(pred == clas_label)
        total += len(clas_label)
        
    if verbose:
        print("Accuracy: ", matches/total)
        print(f"======= {name} Autoencoder method: Finished =======")

    return matches/total

In [28]:
test_autoencoder(df_train, df_test, num_components=128, epochs=1000, autoencoder=nonlinear_autoencoder, verbose=True)

======= nonlinear_autoencoder Autoencoder method: Training and evaluating ... =======
Learning background ...
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 56, 56, 1)]       0         
                                                                 
 sequential_2 (Sequential)   (None, 128)               2817432   
                                                                 
 sequential_3 (Sequential)   (None, 3136)              2820440   
                                                                 
Total params: 5,637,872
Trainable params: 5,637,872
Non-trainable params: 0
_________________________________________________________________
None


InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.